In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
import seaborn as sns
import matplotlib.pyplot as plt

# Modelos
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
import xgboost as xgb


In [4]:
train = pd.read_csv('train_preprocessed.csv')
test = pd.read_csv('test_preprocessed.csv')

X_train = train.drop('FTR', axis=1)
y_train = train['FTR']

X_test = test.drop('FTR', axis=1)
y_test = test['FTR']

In [3]:
def evaluar_modelo(modelo, X_test, y_test, nombre):
    y_pred = modelo.predict(X_test)
    acc = accuracy_score(y_test, y_pred)
    print(f"📊 {nombre} Accuracy: {acc*100:.2f}%\n")
    print(classification_report(y_test, y_pred))
    
    # Matriz de confusión
    cm = confusion_matrix(y_test, y_pred, labels=['H', 'D', 'A'])
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
                xticklabels=['H', 'D', 'A'], 
                yticklabels=['H', 'D', 'A'])
    plt.title(f'Matriz de Confusión - {nombre}')
    plt.ylabel('Real')
    plt.xlabel('Predicho')
    plt.show()


In [5]:
lr = LogisticRegression(multi_class='multinomial', max_iter=1000)
lr.fit(X_train, y_train)

evaluar_modelo(lr, X_test, y_test, 'Regresión Logística')

ValueError: could not convert string to float: 'W'